In [1]:
# importing libraries
import math
import numpy as np
import os
import torch
import torchvision
import tarfile
from torch.utils.data import random_split
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor,ToPILImage
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import seaborn as sns
import torch.nn as nn
import torch.nn.functional as F

In [2]:
sns.set(style="darkgrid")

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
dataset = torchvision.datasets.CIFAR10(root='/gdrive/My Drive/Colab Notebooks/Data_files/cifar10', train=True, download=True, transform=ToTensor())
dataset_test = torchvision.datasets.CIFAR10(root='/gdrive/My Drive/Colab Notebooks/Data_files/cifar10', train=False, transform=ToTensor())

100%|██████████| 170498071/170498071 [00:02<00:00, 65519899.31it/s]


Extracting /gdrive/My Drive/Colab Notebooks/Data_files/cifar10/cifar-10-python.tar.gz to /gdrive/My Drive/Colab Notebooks/Data_files/cifar10


In [5]:
random_seed = 42
torch.manual_seed(random_seed)

In [6]:
val_size = 5000
train_size = len(dataset) - val_size

train_data, val_data = random_split(dataset, [train_size, val_size], generator=torch.manual_seed(random_seed))
len(train_data)

45000

In [7]:
batch_size = 256

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size)

In [8]:
class Neural_Network(nn.Module):
    def __init__(self):
        super(Neural_Network, self).__init__()
        self.fc1 = nn.Linear(32*32*3, 512)
        self.fc2 = nn.Linear(512,256)
        self.fc3 = nn.Linear(256,10)
        
    def forward(self, image):
        image = image.view(-1, 32*32*3)
        
        out = F.relu(self.fc1(image))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        
        return out

In [9]:
my_first_nn = Neural_Network()

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(my_first_nn.parameters(), lr=0.005)

In [12]:
num_epochs = 5

valid_loss_min = np.Inf

for epoch in range(num_epochs):
    
    train_loss, val_loss = 0,0
    
    # Set the mode 
    my_first_nn.train()
    
    for data, label in train_loader:

        # Clear any possible accumulated gradients
        optimizer.zero_grad()

        output = my_first_nn(data)
        
        loss = criterion(output, label)
        # compute grad
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * data.size(0)
        
    # Set the model in eval mode
    my_first_nn.eval()
    
    for data, label in val_loader:
        
        output= my_first_nn(data)
        loss = criterion(output, label)
        val_loss += loss.item() * data.size(0)
        
    train_loss/=train_size
    val_loss/=val_size
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1, 
        train_loss,
        val_loss
        ))
    
    if val_loss<valid_loss_min:
        
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        val_loss))
        torch.save(my_first_nn.state_dict(), 'model.pt')
        valid_loss_min = val_loss

Epoch: 1 	Training Loss: 2.143575 	Validation Loss: 2.120475
Validation loss decreased (inf --> 2.120475).  Saving model ...
Epoch: 2 	Training Loss: 2.089796 	Validation Loss: 2.074820
Validation loss decreased (2.120475 --> 2.074820).  Saving model ...
Epoch: 3 	Training Loss: 2.048068 	Validation Loss: 2.038473
Validation loss decreased (2.074820 --> 2.038473).  Saving model ...
Epoch: 4 	Training Loss: 2.013929 	Validation Loss: 2.007092
Validation loss decreased (2.038473 --> 2.007092).  Saving model ...
Epoch: 5 	Training Loss: 1.985029 	Validation Loss: 1.980506
Validation loss decreased (2.007092 --> 1.980506).  Saving model ...


In [13]:
preds = []

my_first_nn.eval()
    
for data, label in test_loader:

    output= my_first_nn(data)
    
    for tensor in output:
        prediction = torch.argmax(tensor).item()
        preds.append(prediction)

In [14]:
ids = list(range(1, len(preds)+1))
ids.sort(key=lambda x: str(x))

In [15]:
len(preds)

10000

In [16]:
import pandas as pd
df = pd.DataFrame({'id': ids, 'label': preds})
df['label'] = df['label'].apply(lambda x: dataset.classes[x])
df.to_csv('submission.csv', index=False)

In [17]:
test_labels = []

for data,label in dataset_test:
    test_labels.append(label)

In [18]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(test_labels, preds)

In [19]:
acc

0.2968